In [ ]:
%%time

import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip
from pyomo.environ import *
warnings.simplefilter("ignore", category=RuntimeWarning)

In [ ]:
def solve_pyomo_Muller_min2(param_name_str, verbose = False):
    """
    Creates and Solves a Pyomo model for the Muller potential
    
    Parameters:
    -----------
    param_name_str: str, string of parameter names to include. t1 and t2 for CS1 and A,a,b,cx0,and y0 for CS2 Ex: 't1t2' or 'Aabcx0y0'.
    
    Returns:
    --------
    model.obj(): float, The minimum value of the Muller potential for the given sub problem defined by param_name_str
    """
    #Create Model
    model = ConcreteModel()

    # Create a Set to represent the iterable set of variables A1-A4, b1-b4,...y01-y04
    index_set = range(1,5)
    if "A" in param_name_str:
        model.A = Var(Set(initialize=index_set), initialize={1: -210, 2: -100, 3: -200, 4: 10}, 
                  bounds={1: (-300,-100) , 2: (-200,0), 3: (-250,-150), 4: (5,20)})
    else:
        model.A = Param(Set(initialize=index_set), initialize={1: -200, 2: -100, 3: -170, 4: 15})

    if "a" in param_name_str:
        model.a = Var(Set(initialize=index_set), initialize={1: 0, 2: 0, 3: -5, 4: 0},
                 bounds={1: (-1.5,1), 2: (-1.5,1), 3: (-10,-5), 4: (-1,1)})
    else:
        model.a = Param(Set(initialize=index_set), initialize={1: -1, 2: -1, 3: -6.5, 4: 0.7})

    if "b" in param_name_str:
        model.b = Var(Set(initialize=index_set), initialize={1: -1.17, 2: 1.22, 3: 14, 4: -0.5}, 
                 bounds={1: (-2,2), 2: (-2,2), 3: (5,15), 4: (-2,2)})
    else:
        model.b = Param(Set(initialize=index_set), initialize={1: 0, 2: 0, 3: 11, 4: 0.6})

    if "c" in param_name_str:
        model.c = Var(Set(initialize=index_set), initialize={1: -10, 2: -10, 3: -5, 4: 0},
                 bounds={1: (-20,0), 2: (-20,0), 3: (-10,0), 4: (-1,2)})
    else:
        model.c = Param(Set(initialize=index_set), initialize={1: -10, 2: -10, 3: -6.5, 4: 0.7})

    if "x0" in param_name_str:
        model.x0 = Var(Set(initialize=index_set), initialize={1: 0, 2: 0, 3: 0, 4: 0}, 
                  bounds={1: (-2,2), 2: (-2,2), 3: (-2,2), 4: (-2,2)})
    else:
        model.x0 = Param(Set(initialize=index_set), initialize={1: 1, 2: 0, 3: -0.5, 4: -1})

    if "y0" in param_name_str:
        model.y0 = Var(Set(initialize=index_set), initialize={1: 0, 2: 0, 3: 1, 4: 0}, 
                  bounds={1: (-2,2), 2: (-2,2), 3: (0,2), 4: (-2,2)})
    else:
        model.y0 = Param(Set(initialize=index_set), initialize={1: 0, 2: 0.5, 3: 1.5, 4: 1})

    model.x_index = Set(initialize=range(1,3))
    model.x = Var(model.x_index, initialize={1: -1, 2: 0}, bounds={1: (-1.5,1), 2: (-0.5,2)})

    #Define Muller potential
    def calc_muller_pyo(model):  
        #Calculate Muller Potential
        expression = sum((
                model.A[i] * exp(
                model.a[i] * (model.x[1] - model.x0[i]) ** 2 +
                model.b[i] * (model.x[1] - model.x0[i]) * (model.x[2] - model.y0[i]) +
                model.c[i] * (model.x[2] - model.y0[i]) ** 2) for i in range(1,5)))

        return expression
    
    #Define objective
    model.obj = Objective(rule=calc_muller_pyo, sense = minimize)
    
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 10000
    
    try:
        result = solver.solve(model, tee = verbose)
        if verbose:
            # Access solver status and results
            print("Solver Status:", result.solver.status)
            print("Termination Condition:", result.solver.termination_condition)
            # Print the variable value
            #Print model
            model.pprint()

        return model.obj()
    
    except:
        return "fail"

In [ ]:
from itertools import chain, combinations

def all_combinations(parameters):
    all_combinations_list = []
    for r in range(1, len(parameters)+1):
        for subset in combinations(parameters, r):
            combination = ''.join(map(str, subset))
            all_combinations_list.append(combination)
    return all_combinations_list

In [ ]:
parameters = ["A", "a", "b", "c", "x0", "y0"]
all_combinations_list = all_combinations(parameters)
df = pd.DataFrame(all_combinations_list, columns=['param_name_str'])
min_obj = []
for i in range(len(all_combinations_list)):
    model_obj = solve_pyomo_Muller_min2(all_combinations_list[i], verbose = False)
    min_obj.append(model_obj)
    
df["min_value"] = pd.DataFrame(min_obj)

In [ ]:
from IPython.display import display
display(df)
df.to_csv("scaled_mul_v_min.csv", index = False, header = True)

In [ ]:
cs_name_val = 2
meth_name_enum_val = 5
normalize = False
noise_mean = 0
noise_std = 0.01
sep_fact = 1.0
# noise_std = 0.0
# kernel = Kernel_enum(1)
# lenscl = None
# outputscl = None
retrain_GP = 1
reoptimize_obj = 5
seed = 1
num_x_data = 5
num_theta_data_scl = 5
num_theta_data_val = 0
gen_meth_theta_val = None
val_data = None
val_sse_data = None
gen_heat_map_data = True

In [ ]:
#Create System for debugging
#Generate Method (except last part)
CS_name  = CS_name_enum(cs_name_val)
param_name_str = "a"
meth_name = Method_name_enum(meth_name_enum_val)
method = GPBO_Methods(meth_name)
indecies_to_consider = set_idcs_to_consider(cs_name_val, param_name_str)
gen_meth_x = Gen_meth_enum(2) #Note: Has to be the same for validation and sim data
num_theta_data = num_theta_data_scl*len(indecies_to_consider)
print(num_theta_data)
gen_meth_theta = Gen_meth_enum(1)

In [ ]:
print("CS Name: ", CS_name.name)
print(param_name_str)
print(len(indecies_to_consider))

In [ ]:
simulator = simulator_helper_test_fxns(CS_name, indecies_to_consider, noise_mean, noise_std, normalize, seed)
print(simulator.bounds_theta_reg)

#Calculate minimum Muller potential
min_Mul = solve_pyomo_Muller_min2(param_name_str, verbose = False)

#Generate Exp Data
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x)

#Generate Sim Data
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, False)

#Transform Muller Potential
jitter = 1e-5
#Calculate minimum Muller potential
original_muller = sim_data.y_vals
exp_data.y_vals = np.log(exp_data.y_vals - min_Mul + jitter)
sim_data.y_vals = np.log(sim_data.y_vals - min_Mul + jitter)


#Generate sse_sim_data from new sim and exp_data
sim_sse_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, sep_fact, False)

names = [param_name_str + "_" + str(i+1) for i in range(sim_data.theta_vals.shape[1])]
#Define training data based on emulator type
if method.emulator == False:
    train_data = sim_sse_data.theta_vals
    train_y_data = sim_sse_data.y_vals
else:
    train_data = np.concatenate((sim_data.theta_vals, sim_data.x_vals), axis =1)
    train_y_data = sim_data.y_vals
    names += ["x_" + str(i+1) for i in range(sim_data.x_vals.shape[1])]
    
names += ["V", "V_scaled"]
data = np.concatenate((train_data, original_muller.reshape(-1,1), train_y_data.reshape(-1,1)), axis=1)
df_train = pd.DataFrame(data, columns= names)
display(df_train)
csv_name = param_name_str + "_" + str(int(len(train_y_data))) + ".csv"
print(csv_name)
df_train.to_csv(csv_name, index = False, header = True)

In [ ]:
#Preprocess training data
scaler = StandardScaler()
scaler = scaler.fit(train_data)
feat_train_data_scaled = scaler.transform(train_data)

#Create GP Model
#Set GP Model
#Set noise kernel
noise_kern = WhiteKernel(noise_level=noise_std**2, noise_level_bounds= "fixed") #bounds = "fixed"
#Set Constant Kernel
cont_kern = ConstantKernel(constant_value = 1, constant_value_bounds = (1e-3,1e4)) #(1e-3,1e4)
kernel = cont_kern*Matern(length_scale_bounds=(1e-03, 1e3), nu=2.5) + noise_kern #Matern Kernel
#Set initial model lengthscale
kernel.k1.k2.length_scale = np.ones(train_data.shape[1])

gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, n_restarts_optimizer=retrain_GP, 
                                    random_state = seed, optimizer = "fmin_l_bfgs_b", normalize_y = True)

#Fit GP Model
# scaler = preprocessing.StandardScaler().fit(X_train)
fit_gp_model = gp_model.fit(feat_train_data_scaled, train_y_data)
print(fit_gp_model.kernel_)

In [ ]:
#Create Heat Map validation data

#Create list of heat map theta data
heat_map_data_dict = {}

#Create a linspace for the number of dimensions and define number of points
dim_list = np.linspace(0,simulator.dim_theta-1,simulator.dim_theta)

#Create a list of all combinations (without repeats e.g no (1,1), (2,2)) of dimensions of theta
mesh_combos = np.array(list(combinations(dim_list, 2)), dtype = int)

n_points = 20

#Meshgrid set always defined by n_ponts**2
theta_set = np.tile(np.array(simulator.theta_true), (n_points**2, 1))

#Unnormalize x_vals if necessary
norm_x_vals = exp_data.x_vals
if normalize == True:
    lower_bound = simulator.bounds_x[0]
    upper_bound = simulator.bounds_x[1]
    norm_x_vals = norm_x_vals*(upper_bound - lower_bound) + lower_bound

#Infer how many times to repeat theta and x values given that heat maps are meshgrid form by definition
#The meshgrid of parameter values created below is symmetric, therefore, x is repeated by n_points**2 for a 2D meshgrid
repeat_x = n_points**2 #Square because only 2 values at a time change
x_vals = np.vstack([norm_x_vals]*repeat_x)
repeat_theta = exp_data.get_num_x_vals()

#Loop over all possible theta combinations of 2
for i in range(len(mesh_combos)):
    #Create a copy of the true values to change the mehsgrid valus on
    theta_set_copy = np.copy(theta_set)
    #Set the indeces of theta_set for evaluation as each row of mesh_combos
    idcs = mesh_combos[i]
    #define name of parameter set as tuple ("param_1,param_2")
    data_set_name = (simulator.theta_true_names[idcs[0]], simulator.theta_true_names[idcs[1]])

    #Create a meshgrid of values of the 2 selected values of theta and reshape to the correct shape
    #Assume that theta1 and theta2 have equal number of points on the meshgrid
    theta1 = np.linspace(simulator.bounds_theta_reg[0][idcs[0]], simulator.bounds_theta_reg[1][idcs[0]], n_points)
    theta2 = np.linspace(simulator.bounds_theta_reg[0][idcs[1]], simulator.bounds_theta_reg[1][idcs[1]], n_points)
    theta12_mesh = np.array(np.meshgrid(theta1, theta2))
    theta12_vals = np.array(theta12_mesh).T.reshape(-1,2)

    #Set initial values for evaluation (true values) to meshgrid values
    theta_set_copy[:,idcs] = theta12_vals

    #Put values into instance of data class
    #Create data set based on emulator status
    if method.emulator == True:
        #Repeat the theta vals for Type 2 methods to ensure that theta and x values are in the correct form for evaluation with gp_emulator.eval_gp_mean_heat_map()
        theta_vals =  np.repeat(theta_set_copy, repeat_theta , axis =0)
        data_set = Data(theta_vals, x_vals, None,None,None,None,None,None, simulator.bounds_theta_reg, simulator.bounds_x, sep_fact, seed)
    else:
        data_set = Data(theta_set_copy, norm_x_vals, None,None,None,None,None,None, simulator.bounds_theta_reg, simulator.bounds_x, sep_fact, seed)
#             #normalize values between 0 and 1 if necessary
    if normalize == True:
        data_set = data_set.norm_feature_data()
    #Append data set to dictionary with name
    heat_map_data_dict[data_set_name] = data_set

In [ ]:
#Find minimum sse point (later)

In [ ]:
#Make Heat Map Data
pair_id = 0
log_data = True
#Get pair ID
if isinstance(pair_id, str):
    param_names = pair_id
elif isinstance(pair_id, int):
    param_names = list(heat_map_data_dict.keys())[pair_id]
else:
    raise Warning("Invalid pair_id!")

heat_map_data = heat_map_data_dict[param_names]
#Define heat map evaluation data based on emulator type
if method.emulator == False:
    featurized_hm_data = heat_map_data.theta_vals
else:
    featurized_hm_data = np.concatenate((heat_map_data.theta_vals, heat_map_data.x_vals), axis =1)

#Get index of param set
idcs_to_plot = [simulator.theta_true_names.index(name) for name in param_names]   
theta_true = simulator.theta_true
train_theta = train_data

#Calculate GP mean and var for heat map data
#Scale data
feat_hm_data_scaled = scaler.transform(featurized_hm_data)
heat_map_data.gp_mean, heat_map_data.gp_var = fit_gp_model.predict(feat_hm_data_scaled, return_std=True)

#If not in emulator form, rearrange the data such that y_sim can be calculated
if method.emulator == False:
    #Rearrange the data such that it is in emulator form
    n_points = int(np.sqrt(heat_map_data.get_num_theta())) #Since meshgrid data is always in meshgrid form this gets num_points/param
    repeat_x = n_points**2 #Square because only 2 values at a time change
    x_vals = np.vstack([exp_data.x_vals]*repeat_x) #Repeat x_vals n_points**2 number of times
#         print(x_vals)
    repeat_theta = exp_data.get_num_x_vals() #Repeat theta len(x) number of times
    theta_vals =  np.repeat(heat_map_data.theta_vals, repeat_theta , axis =0) #Create theta data repeated
    #Generate full data class
    heat_map_data = Data(theta_vals, x_vals, None, heat_map_data.gp_mean, heat_map_data.gp_var,None,None,None,
                         simulator.bounds_theta_reg, simulator.bounds_x, sep_fact, seed)

#Calculate y and sse values
heat_map_data.y_vals = simulator.gen_y_data(heat_map_data, 0 , 0)
heat_map_data.y_vals = np.log(heat_map_data.y_vals - min_Mul + jitter)
heat_map_sse_data = simulator.sim_data_to_sse_sim_data(method, heat_map_data, exp_data, sep_fact, gen_val_data = False)

#Calculate SSE, SSE var, and EI with GP
if method.emulator == False:
    heat_map_data.sse = heat_map_data.gp_mean
    heat_map_data.sse_var = heat_map_data.gp_var          
else:
#Find length of theta and number of unique x in data arrays
    len_theta = heat_map_data.get_num_theta()
    len_x = len(heat_map_data.get_unique_x())

    #Make sse arrays as an empty lists. Will add one value for each training point
    sse_mean = []
    sse_var = []

    #Iterates over evey combination of theta to find the sse for each combination
    #Note to do this Xexp and X **must** use the same values
    if len_theta > 0: #Only do this if you actually have data
        for i in range(0, len_theta, len_x):
            sse_mean.append( sum((heat_map_data.gp_mean[i:i+len_x] - exp_data.y_vals)**2) ) #Vector 
            error_points_sq = (2*(heat_map_data.gp_mean[i:i+len_x] - exp_data.y_vals))**2 #Vector
            #sse_var = (2*(GP -y ))**2 * var
            sse_var.append( (error_points_sq@heat_map_data.gp_var[i:i+len_x]) ) #This SSE_variance CAN'T be negative

    #Lists to arrays
    sse_mean = np.array(sse_mean)
    sse_var = np.array(sse_var)


    #For Method 2B, make sse and sse_var data in the log form
    if method.obj.value == 2:
        #Propogation of errors: stdev_ln(val) = stdev/val
        sse_var = sse_var/(sse_mean**2)
        #Set mean to new value
        sse_mean = np.log(sse_mean)


    #Set class parameters
    heat_map_data.sse = sse_mean
    heat_map_data.sse_var = sse_var

#Get log or unlogged data values        
if log_data == False:
    #Change sse sim, mean, and stdev to not log for 1B and 2B
    if method.obj.value == 2:
        #SSE variance is var*(e^((log(sse)))^2
        heat_map_data.sse = np.exp(heat_map_data.sse)
        heat_map_data.sse_var = heat_map_data.sse_var*heat_map_data.sse**2            
        heat_map_sse_data.y_vals = np.exp(heat_map_sse_data.y_vals)

#If getting log values
else:
    #Get log data from 1A, 2A, and 2C
    if method.obj.value == 1:            
        #SSE Variance is var/sse**2
        heat_map_data.sse_var = heat_map_data.sse_var/heat_map_data.sse**2
        heat_map_data.sse = np.log(heat_map_data.sse)
        heat_map_sse_data.y_vals = np.log(heat_map_sse_data.y_vals)

#Create test mesh
#Define original theta_vals (for restoration later)
org_theta = heat_map_data.theta_vals
#Redefine the theta_vals in the given Data class to be only the 2D (varying) parts you want to plot
heat_map_data.theta_vals = heat_map_data.theta_vals[:,idcs_to_plot]
#Create a meshgrid with x and y values fron the uniwue theta values of that array
unique_theta = heat_map_data.get_unique_theta()
theta_pts = int(np.sqrt(len(unique_theta)))
test_mesh = unique_theta.reshape(theta_pts,theta_pts,-1).T
heat_map_data.theta_vals = org_theta

sse_sim = heat_map_sse_data.y_vals.reshape(theta_pts,theta_pts).T
sse_mean = heat_map_data.sse.reshape(theta_pts,theta_pts).T
sse_var = heat_map_data.sse_var.reshape(theta_pts,theta_pts).T

all_data = [sse_sim, sse_mean, sse_var]

#Scale theta true if necessary
if normalize == True:
    lower_bound = simulator.bounds_theta_reg[0]
    upper_bound = simulator.bounds_theta_reg[1]
    theta_true = (theta_true - lower_bound) / (upper_bound - lower_bound)
    
theta_opt = unique_theta[np.argmin(sse_mean.T)]
print(theta_opt)

In [ ]:
#Plot HM Data
#Plot Heat maps
#Make heat maps
title_fontsize = 24
other_fontsize = 20
xbins = 4
ybins = 5
zbins = 900
save_fig = False
cmap = "autumn"

title = "Heat Map Pair " + "-".join(map(str, param_names[0]))
title = None
#     z = [sse_sim, sse_mean, sse_var, ei]
#     z_titles = ["ln(sse_sim)", "ln(sse)", "ln(sse_var)", "log(ei)"]
#     levels = [100,100,100, 100]

z = [sse_sim, sse_mean, sse_var]
z_titles = ["ln(sse_sim)", "ln(sse)", "len(sse_var)"]
levels = [100,100,100]
#     z = [sse_mean]
#     z_titles = [meth_names[i] + " log("+ r"$\mathbf{e(\theta)}$" + ")"]

if save_fig == False:
    save_path = None

plot_heat_maps(test_mesh, theta_true, theta_opt, None, train_theta, param_names, levels, idcs_to_plot, 
               z, z_titles, xbins, ybins, zbins, title, title_fontsize, other_fontsize, cmap, save_path)